In [1]:
pwd

u'C:\\rahuketu\\programming\\Learning_C'

#Check if csc compiler path has been setup


In [2]:
ip = get_ipython()
ip.system("csc")

Microsoft (R) Visual C# Compiler version 4.6.0057.0

for Microsoft (R) .NET Framework 4.5
Copyright (C) Microsoft Corporation. All rights reserved.

warning CS2008: No source files specified
error CS1562: Outputs without source must have the /out option specified


In [3]:
%%writefile comp_utils.py

from __future__ import print_function

import contextlib2
import sys
import os
import time
import functools

from colorama import init

try:
    
    from termcolor import colored

    # use Colorama to make Termcolor work on Windows too
    init()
except:
    def colored(cmd, fg, bg):
        print("No module named colored")
        return cmd

@contextlib2.contextmanager
def redirect_stdout(stream):
    old_stdout, sys.stdout = sys.stdout, stream
    try:
        yield
    finally:
        sys.stdout = old_stdout


@contextlib2.contextmanager       
def cd(name):
    current_dir = os.getcwd()
    print(name)
    if os.path.isdir(name):
        print("Dir")
        nam = name
    elif os.path.isfile(name):
        nam = os.path.dirname(os.path.abspath(name))
    else:
        raise IOError()
    os.chdir(nam)
    yield 
    os.chdir(current_dir)


def echo(cmd, colorfg=None, colorbg=None):
    print(colored(cmd, colorfg, colorbg))


def echo_header(cmd):
    echo("----------{0}----------".format(cmd))


def clockit(fn):
    '''Timeit function like this doesnot work with the thread calls'''
    @functools.wraps(fn)
    def wrapper(*args,**kwargs):
        echo_header("Starting %s" % fn.__name__)
        start = time.clock()
        result = fn(*args,**kwargs)
        end = time.clock()
        echo_header('Time Elapsed :%s: %.3f seconds' %( fn.__name__, round(end-start,3)))
        return result
    return wrapper



Overwriting comp_utils.py


In [4]:
## Testing comp_utils

from __future__ import print_function

#Testing
import comp_utils
from comp_utils import redirect_stdout, clockit, cd, echo

try:
    from StringIO import StringIO
except:
    from io import StringIO

output = StringIO()
output.write(u"Hello")
print(output.getvalue())
output.close()


def test():
    output2 = StringIO()
    with redirect_stdout(output2):
        print("Hello3/")
    print("Getting Output")
    print(output2.getvalue())
test()


@clockit
def ec(sd):
    print(sd)

ec("SSS")

Hello
Getting Output
Hello3/

----------Starting ec----------
SSS
----------Time Elapsed :ec: 0.000 seconds----------


In [5]:
%%writefile compiler_ipy.py

from __future__ import print_function
from comp_utils import redirect_stdout, clockit, cd, echo
from IPython.core.magic import register_line_magic, register_cell_magic
from contextlib2 import closing
try:
    from StringIO import StringIO
except:
    from io import StringIO
    
import os

ip = get_ipython()

CSC = "csc"
#CSC_path = "C:\Windows\Microsoft.NET\Framework\v4.0.30319\csc.exe"

@register_line_magic
@clockit
def cscexy(csfile):
    
    executable = os.path.splitext(os.path.basename(csfile))[0] + '.exe'
    echo("Building executable", "red")
    command = CSC + " " + str(csfile)
    echo(">>" + command, "red")
    ip.system(command)
    echo("Running executable", "red")
    echo(">>" + executable, "red")
    with closing(StringIO()) as output:
        with redirect_stdout(output):
            ip.system(executable)
        echo('\n'+output.getvalue() , "blue" )
        
@register_cell_magic
def cscexy(csfile, cell):
    ip.run_cell_magic('writefile', csfile, cell)
    ip.run_line_magic('cscexy', csfile)
        
#cscexy(None)

Overwriting compiler_ipy.py


In [6]:
%run compiler_ipy.py

#SetUp Compiler Path for C# #

In [8]:
%%writefile hello_world.cs
public class Hello1
{
    public static void Main()
    {
        System.Console.WriteLine("Hello, World from c#1:::");
    }
}

Overwriting hello_world.cs


In [9]:
%cscexy hello_world.cs

----------Starting cscexy----------
Building executable
>>csc hello_world.cs
Microsoft (R) Visual C# Compiler version 4.6.0057.0

for Microsoft (R) .NET Framework 4.5
Copyright (C) Microsoft Corporation. All rights reserved.

Running executable
>>hello_world.exe

Hello, World from c#1:::

----------Time Elapsed :cscexy: 2.330 seconds----------


In [10]:
!hello_world.exe

Hello, World from c#1:::


In [12]:
%%cscexy hello_world2.cs
public class Hello1
{
    public static void Main()
    {
        System.Console.WriteLine("Hello, World from direct C#2");
    }
}

Overwriting hello_world2.cs
----------Starting cscexy----------
Building executable
>>csc hello_world2.cs
Microsoft (R) Visual C# Compiler version 4.6.0057.0

for Microsoft (R) .NET Framework 4.5
Copyright (C) Microsoft Corporation. All rights reserved.

Running executable
>>hello_world2.exe

Hello, World from direct C#2

----------Time Elapsed :cscexy: 1.383 seconds----------


In [13]:
%%cscexy hello_world3.cs
public class Hello1
{
    public static void Main()
    {
        System.Console.WriteLine("Hello, World from direct C#3");
    }
}

Overwriting hello_world3.cs
----------Starting cscexy----------
Building executable
>>csc hello_world3.cs
Microsoft (R) Visual C# Compiler version 4.6.0057.0

for Microsoft (R) .NET Framework 4.5
Copyright (C) Microsoft Corporation. All rights reserved.

Running executable
>>hello_world3.exe

Hello, World from direct C#3

----------Time Elapsed :cscexy: 1.239 seconds----------
